In [9]:
import pandas as pd

In [35]:
def cal_coastline(df, lat_col_name, lng_col_name):
    """
    calculate the distance between house and coast line
    df: the dataframe that need to calculate the distance between coastline and house
    lat_col_name, lng_col_name: the column name of Latitude and Longitude
    """
    
    import json
    from geopy.distance import distance

    # ---------------------------- arranage coordinate from geojson ----------------------------
    # import coastline geojson
    with open("../dataset/Coastline.geojson", "r") as f:
        json_file = json.load(f)

    json_file = json_file["features"][0]["geometry"]["coordinates"]

    coastline = [] # create an empty list for calculation
    # arrange the lat and long in the list and put it as a tuple in coastline list for calcualtion
    for coordinate in json_file:
        lat = coordinate[1]
        lng = coordinate[0]
        coastline.append(tuple([lat,lng])) 
    # ---------------------------- arranage coordinate from geojson ----------------------------

    # ---------------------------- coastal distance calculation ----------------------------
    coast_distance = [] # store the calculated coastal distance

    for index, row in df.iterrows():
        row_ls = [] # temp store for all the distance between coastal distance
        for coastline_coordinate in coastline:
            
            house_coordinate = tuple([row[lat_col_name],row[lng_col_name]])
            # calculate the distance from coast
            dum_dis = distance(coastline_coordinate, house_coordinate).m

            row_ls.append(dum_dis)
        # calculate the shortest distance between house and coastline and append into the coast distance
        coast_distance.append(min(row_ls))
    # ---------------------------- coastal distance calculation ----------------------------

    return coast_distance

In [36]:
def property_mapping(df, col_name):
    """
    mapping the property type with number rank based on the median price
    """
    property_type_map = {
        'house'                 : 1,
        'duplex-semi-detached'  : 2,
        'villa'                 : 3,
        'townhouse'             : 4,
        'terrace'               : 5,
        'apartment'             : 6,
        'flat'                  : 6,  # Same as apartment
        'unit'                  : 6,  # Same as apartment
        'acreage'               : 7,  # Based on land median
        'residential-other'     : 8
    }

    # Apply the mapping
    df[col_name] = df[col_name].map(property_type_map)

    return df

In [ ]:
raw_dataset = pd.read_csv("../dataset/perth_property_data.csv", chunksize=5000)

df = pd.DataFrame()
for chunk_df in raw_dataset:

    chunk_df = property_mapping(chunk_df, "Property_Type")

    coast_distance = cal_coastline(chunk_df, "Latitude", "Longitude")
    chunk_df["Distance_to_Coast"] = coast_distance

    # convert to datetime type
    chunk_df["Date_Sold"] = pd.to_datetime(chunk_df["Date_Sold"], format="%d/%m/%Y")

    # append the complete dataframe
    df = pd.concat([df, chunk_df], axis = 0, ignore_index=True)

df

,Listing_ID,Price,Agency_Name,Postcode,Address,Suburb,Longitude,Latitude,Property_Type,Bedrooms,Bathrooms,Parking_Spaces,Date_Sold,Land_Size,Primary_School_Name,Primary_School_Distance,Secondary_School_Name,Secondary_School_Distance,Distance_to_CBD,Distance_to_Coast
0,142678556,880000,Porter Matthews Metro - Belmont,6076,64 Basildon Road,Lesmurdie,116.033540,-32.007450,1,3,2,5,2023-08-23,1533,LESMURDIE PRIMARY SCHOOL,1149,MAZENOD COLLEGE,508,17721,26634.585679
1,121988994,3850000,William Porteous Properties International Pty ...,6011,35 View Street,Peppermint Grove,115.767527,-32.001289,1,5,3,3,2016-06-21,1532,ST HILDA'S ANGLICAN SCHOOL FOR GIRLS,418,ST HILDA'S ANGLICAN SCHOOL FOR GIRLS,418,9983,1539.284806
2,135695670,5890000,William Porteous Properties International Pty ...,6011,33 Mcneil Street,Peppermint Grove,115.770700,-31.993860,1,5,3,2,2021-12-17,1529,PRESBYTERIAN LADIES COLLEGE,548,PRESBYTERIAN LADIES COLLEGE,548,9307,1864.233909
3,131680046,430000,David Mattock Real Estate,6056,33 Harper Street,Woodbridge,115.996020,-31.887670,1,4,1,0,2019-12-09,1526,WOODBRIDGE PRIMARY SCHOOL,569,GOVERNOR STIRLING SENIOR HIGH SCHOOL,697,15035,22844.395654
4,141059588,900000,Next Vision Real Estate - COCKBURN CENTRAL,6163,46 Davilak Avenue,Hamilton Hill,115.770110,-32.084890,1,3,2,4,2023-05-29,1524,KERRY STREET COMMUNITY SCHOOL,433,FREMANTLE CHRISTIAN COLLEGE,484,16724,1519.903819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42953,131703414,10000,Independent,6152,14 Sawyer Way,Salter Point,115.867510,-32.021895,1,4,3,2,2019-11-15,315,ST PIUS X CATHOLIC SCHOOL,678,AQUINAS COLLEGE,722,7647,10968.150572
42954,138928707,7300,Shared-Space Property Agency - HILLARYS,6026,73 Dalmain Street,Kingsley,115.789870,-31.814920,1,4,2,3,2022-07-02,680,DALMAIN PRIMARY SCHOOL,136,ST STEPHEN'S SCHOOL,998,16630,4526.378145
42955,129409222,1100,City to Surf Real Estate - DARCH,6077,100 Colonial Circle,Gnangara,115.857283,-31.784058,1,5,2,2,2019-05-15,11100,LANDSDALE PRIMARY SCHOOL,2195,LANDSDALE CHRISTIAN SCHOOL,2689,18790,11739.228474
42956,107492460,820,Independent,6164,Lot 48 LOUISIANA GLEN,Banjup,115.900034,-32.119491,1,3,3,0,2011-10-08,20003,PIARA WATERS PRIMARY SCHOOL,1778,HARRISDALE SENIOR HIGH SCHOOL,3380,18848,12932.518667


In [39]:
icsea_df = pd.read_csv("../dataset/school_ICSEA.csv")
icsea_df.head(2)

,School_Name,Type,ICSEA
0,LESMURDIE PRIMARY SCHOOL,P,1043
1,ST HILDA'S ANGLICAN SCHOOL FOR GIRLS,P,1197


In [40]:
second_icsea_df = icsea_df.loc[(icsea_df["Type"] == "S"), :]
primary_icsea_df = icsea_df.loc[(icsea_df["Type"] == "P"), :]

In [50]:
# merge secondary school ICSEA data included into this dataset
full_df = pd.merge(df, second_icsea_df, how = "left", right_on = "School_Name", left_on = "Secondary_School_Name")
full_df = full_df.rename(columns= {"ICSEA" : "Secondary_ICSEA"}).drop(columns = ["School_Name", "Type"])

# merge primary school ICSEA data included into this dataset
full_df = pd.merge(full_df, primary_icsea_df, how = "left", right_on = "School_Name", left_on = "Primary_School_Name") 
full_df = full_df.rename(columns= {"ICSEA" : "Primary_ICSEA"}).drop(columns = ["School_Name", "Type"])

In [53]:
full_df.to_csv("./output/complete_dataset.csv", index = False)